In [2]:
import keras
from keras.models import Sequential, Model,load_model
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.applications.vgg16 import VGG16
import numpy as np
from IPython.display import Image
from keras.layers.core import Flatten, Dense, Dropout
from sklearn.utils import shuffle
import matplotlib.pylab as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
#Montagem do drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#mapeando area de trabalho
import os
os.chdir('/content/drive/My Drive/datasets/jetski/jet')
!ls

modeloJet.h5  model.tflite  modelV2  modelV3  test  train


In [5]:
#carregando uma VGG sem a parte dense
model= VGG16(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
# congelar as camadas de convnet
for layer in model.layers:
	layer.trainable = False

model.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [6]:
# criando a parte densa
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(12, activation='relu')(flat1)
class2 = Dense(8, activation='relu')(class1)
output = Dense(3, activation='softmax')(class2)
# define new model
modelFinal = Model(inputs=model.inputs, outputs=output)

In [7]:
modelFinal.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
#função para carregar as imagens de treino/val
def creating_traning_data():
  traningDataX=[]
  traningDataY=[]
  os.chdir('/content/drive/My Drive/datasets/jetski/jet/train')
  for img in os.listdir():
    img=image.load_img(img, target_size=(224,224,3));
    img_array=image.img_to_array(img).astype('float32')/255
    traningDataX.append(img_array)
    traningDataY.append('jet')

  os.chdir('/content/drive/My Drive/datasets/jetski/lixo/train')
  for img in os.listdir():
    img=image.load_img(img, target_size=(224,224,3))
    img_array=image.img_to_array(img).astype('float32')/255
    traningDataX.append(img_array)
    traningDataY.append('lixo')

  os.chdir('/content/drive/My Drive/datasets/jetski/banhista/train')
  for img in os.listdir():
    img=image.load_img(img, target_size=(224,224,3))
    img_array=image.img_to_array(img).astype('float32')/255
    traningDataX.append(img_array)
    traningDataY.append('banhista')

  os.chdir('/content/drive/My Drive/datasets/jetski/jet')

  #onehot encoding nas labels.
  le = LabelEncoder()
  traningDataY = le.fit_transform(traningDataY)
  traningDataY = to_categorical(traningDataY, 3)

  traningDataX = np.array(traningDataX)
  traningDataY = np.array(traningDataY)

  traningDataX,traningDataY = shuffle(traningDataX,traningDataY)

  return [traningDataX, traningDataY]


In [ ]:
# Carregando as imagens
[traningDataX, traningDataY] = creating_traning_data()

print(len(traningDataX))

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
np.shape(traningDataY)

In [ ]:
#inicializando o data generator
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

In [ ]:
#preparando treino e val
(trainX, valX, trainY, valY) = train_test_split(traningDataX, traningDataY, test_size=0.2, random_state=42)

In [ ]:
np.shape(valX)

In [ ]:
h = modelFinal.fit(x=aug.flow(trainX, trainY, batch_size=10), validation_data=(valX, valY),steps_per_epoch=len(trainX)/10,epochs=30)

In [ ]:
plt.plot(h.history['accuracy'])
plt.plot(h.history['val_accuracy'])

In [ ]:
#função para carregar as imagens de treino/val
def creating_test_data():
  testDataX=[]
  testDataY=[]
  os.chdir('/content/drive/My Drive/datasets/jetski/jet/test')
  for img in os.listdir():
    img=image.load_img(img, target_size=(224,224,3));
    img_array=image.img_to_array(img).astype('float32')/255
    testDataX.append(img_array)
    testDataY.append('jet')

  os.chdir('/content/drive/My Drive/datasets/jetski/lixo/test')
  for img in os.listdir():
    img=image.load_img(img, target_size=(224,224,3))
    img_array=image.img_to_array(img).astype('float32')/255
    testDataX.append(img_array)
    testDataY.append('lixo')

  os.chdir('/content/drive/My Drive/datasets/jetski/banhista/test')
  for img in os.listdir():
    img=image.load_img(img, target_size=(224,224,3))
    img_array=image.img_to_array(img).astype('float32')/255
    testDataX.append(img_array)
    testDataY.append('banhista')

  

  os.chdir('/content/drive/My Drive/datasets/jetski')

  #onehot encoding nas labels.
  le = LabelEncoder()
  testDataY = le.fit_transform(testDataY)
  testDataY = to_categorical(testDataY, 3)

  testDataX = np.array(testDataX)
  testDataY = np.array(testDataY)

  testDataX,testDataY = shuffle(testDataX,testDataY)

  return [testDataX, testDataY]

In [ ]:
testDataX, testDataY  = creating_test_data()

In [ ]:
# evaluate the keras model
accuracy = modelFinal.evaluate(testDataX, testDataY)

In [ ]:
accuracy

In [ ]:
plt.imshow(testDataX[6], interpolation='nearest')
plt.show()

In [ ]:
#testando a saida da VGG
X=testDataX[6].reshape((1,224,224,3))
X=preprocess_input(X)
saida=modelFinal.predict(X)
saida[0]

In [ ]:
os.chdir('/content/drive/My Drive/datasets/jetski/jet')
modelFinal.save('modelV4.h5')